In [1]:
!pip install sacrebleu -q
!pip install datasets -q
!pip install tqdm -q
!pip install numpy -q
!pip install huggingface_hub -q
!pip install faster-whisper -q
!pip install transformers -q
!pip install evaluate -q
!pip install jiwer -q

In [2]:
from datasets import load_dataset, Dataset, Audio, DatasetDict, IterableDatasetDict
import pandas as pd
from tqdm import tqdm
from librosa import load, get_duration
from tqdm.notebook import tqdm
from transformers import WhisperFeatureExtractor, WhisperForConditionalGeneration, WhisperTokenizer, WhisperProcessor, Seq2SeqTrainingArguments, Seq2SeqTrainer, Trainer
import torch
import torch
from torch.utils.data import IterableDataset
from dataclasses import dataclass
from typing import Any, Dict, List, Union
import evaluate

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [24]:
import os
cache_dir = "/content/huggingface_cache"
os.makedirs(cache_dir, exist_ok=True)

# Set ALL Hugging Face related cache directories
os.environ["TRANSFORMERS_CACHE"] = os.path.join(cache_dir, "transformers")
os.environ["HF_DATASETS_CACHE"] = os.path.join(cache_dir, "datasets")
os.environ["HF_HOME"] = os.path.join(cache_dir, "hf_home")
os.environ["HF_ASSETS_CACHE"] = os.path.join(cache_dir, "assets")
os.environ["HUGGINGFACE_HUB_CACHE"] = os.path.join(cache_dir, "hub")
os.environ["HF_MODULES_CACHE"] = os.path.join(cache_dir, "modules")

# Create all directories
for dir_path in [os.environ["TRANSFORMERS_CACHE"],
                os.environ["HF_DATASETS_CACHE"],
                os.environ["HF_HOME"],
                os.environ["HF_ASSETS_CACHE"],
                os.environ["HUGGINGFACE_HUB_CACHE"],
                os.environ["HF_MODULES_CACHE"]]:
    os.makedirs(dir_path, exist_ok=True)

# Force datasets to use the new cache
from datasets import config
config.HF_DATASETS_CACHE = os.environ["HF_DATASETS_CACHE"]

In [67]:
train_dataset = load_dataset("kreasof-ai/be-en-IWSLT2025",split="train[:1000]")

Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

In [68]:
test_dataset = load_dataset("kreasof-ai/be-en-IWSLT2025", split="test[:200]")

Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

In [69]:
train_dataset

Dataset({
    features: ['audio', 'sentence', 'translation', 'speaker_id'],
    num_rows: 1000
})

In [70]:
test_dataset

Dataset({
    features: ['audio', 'sentence', 'translation', 'speaker_id'],
    num_rows: 200
})

In [71]:
  def prepare_dataset(batch):
      audio = batch["audio"]

      #compute log-Me1 input features from input audio array
      batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

      # encode target text to label ids
      batch["labels"] = tokenizer(batch["sentence"],
                                  max_length=448,  # Ensure target length is within model limits
                                  truncation=True,
                                  padding="max_length").input_ids
      return batch

In [72]:
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

In [73]:
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="en", task="transcribe")

In [74]:
train_converted = train_dataset.map(prepare_dataset, remove_columns=train_dataset.column_names, num_proc=1)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [75]:
test_converted = test_dataset.map(prepare_dataset, remove_columns=test_dataset.column_names, num_proc=1)

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [76]:
train_dataset

Dataset({
    features: ['audio', 'sentence', 'translation', 'speaker_id'],
    num_rows: 1000
})

In [77]:
output_dir = "./"
hf_repo="kreasof-ai"
output_dir="dummy"

In [78]:
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [79]:
processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="en", task="transcribe")

In [80]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [94]:
metric_bleu = evaluate.load("sacrebleu")
metric_chrf = evaluate.load("chrf")
metric_wer = evaluate.load("wer")
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    # wer = 100 * metric_wer.compute(predictions=pred_str, references=label_str)

    # bleu = metric_bleu.compute(predictions=pred_str, references=label_str)
    # bleu = round(bleu["score"], 2)

    chrf = metric_chrf.compute(predictions=pred_str, references=label_str)
    chrf = round(chrf["score"], 2)

    return {"chrf": chrf}

In [82]:
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
model.generation_config.language = "en"
model.generation_config.task = "transcribe"

# model.generation_config.forced_decoder_ids = None

In [120]:
max_steps = 300
training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,  # Change this to your desired output directory
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # Increase by 2x for every 2x decrease in batch size
    num_train_epochs=3,
    gradient_checkpointing=True,

    bf16=True,
    # hub_model_id=f"{hf_repo}/{output_dir}",
    # Changed to evaluate at the end of each epoch
    evaluation_strategy="steps",
    eval_steps=50,
    save_steps=50,

    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    max_steps=max_steps,
    # Save best model based on WER metric at the end of each epoch
    save_strategy="steps",
    load_best_model_at_end=True,
    metric_for_best_model="chrf",
    greater_is_better=True,  # Lower WER is better, for CHRF greater is better

    # More frequent logging for better monitoring
    logging_steps=1,  # Adjust if needed
    report_to=["tensorboard"],

    # push_to_hub=True,  # Upload model to Hugging Face Hub if needed
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [121]:
def model_init():
    return WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")


In [122]:
trainer = Seq2SeqTrainer(
    args=training_args,
    model_init= model_init,
    # model=model,
    train_dataset=train_converted,
    eval_dataset=test_converted,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

<ipython-input-122-c2660467cef7>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


# Optuna

In [46]:
!pip install optuna


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 15.2 MB/s eta 0:00:00


In [123]:
def objective(metrics):
    print(metrics)
    return metrics["eval_chrf"]

In [124]:
def hp_space(trial):
    return {
        "learning_rate": trial.suggest_loguniform("learning_rate", 1e-5, 3e-5),
        "warmup_steps": trial.suggest_categorical("warmup_steps", [0, 25, 50, 100]),
    }

best_run = trainer.hyperparameter_search(
    direction="maximize",  # Minimize validation loss
    backend="optuna",
    hp_space=hp_space,
    n_trials=3,
    compute_objective=objective,
)

print("Best hyperparameters:", best_run.hyperparameters)


[I 2025-04-02 14:16:57,313] A new study created in memory with name: no-name-3c940c16-fab2-469e-ae06-d3684258e008
<ipython-input-124-cba6e771ef92>:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "learning_rate": trial.suggest_loguniform("learning_rate", 1e-5, 3e-5),


Step,Training Loss,Validation Loss,Chrf
50,0.121000,0.137935,36.360000
100,0.074300,0.093687,75.890000
150,0.045200,0.076049,77.940000
200,0.026100,0.072177,79.460000
250,0.038500,0.072081,78.760000
300,0.024700,0.072935,78.130000


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr

{'eval_loss': 0.1379351019859314, 'eval_chrf': 36.36}


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecate

...

{'eval_loss': 0.0709715336561203, 'eval_chrf': 80.51}


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr

{'eval_loss': 0.0721684917807579, 'eval_chrf': 80.19}


There were missing keys in the checkpoint model loaded: ['proj_out.weight'].
[I 2025-04-02 15:34:56,551] Trial 2 finished with value: 80.19 and parameters: {'learning_rate': 2.608782688782621e-05, 'warmup_steps': 25}. Best is trial 2 with value: 80.19.


Best hyperparameters: {'learning_rate': 2.608782688782621e-05, 'warmup_steps': 25}


In [125]:
print("Best hyperparameters:", best_run.hyperparameters)

Best hyperparameters: {'learning_rate': 2.608782688782621e-05, 'warmup_steps': 25}


In [118]:
import gc

gc.collect()
torch.cuda.empty_cache()


In [ ]:
# import optuna
# from transformers import Seq2SeqTrainer, TrainingArguments
# from transformers import WhisperForConditionalGeneration
# from datasets import load_dataset

# # Load your datasets
# train_dataset = load_dataset("your_dataset_name", split="train")
# eval_dataset = load_dataset("your_dataset_name", split="validation")

# # Define the objective function for Optuna
# def objective(trial):
#     # Hyperparameter search space
#     learning_rate = trial.suggest_loguniform("learning_rate", 2e-5, 3e-5)  # Suggested learning rate range
#     warmup_steps = trial.suggest_categorical("warmup_steps", [1000, 2000])  # Suggested warmup steps options

#     # Define your training arguments
#     training_args = TrainingArguments(
#         output_dir="./results",  # Output directory for model checkpoints
#         evaluation_strategy="epoch",  # Evaluate after each epoch
#         save_strategy="epoch",  # Save model after each epoch
#         learning_rate=learning_rate,  # Learning rate from Optuna
#         warmup_steps=warmup_steps,  # Warmup steps from Optuna
#         num_train_epochs=5,  # Number of epochs
#         per_device_train_batch_size=8,  # Training batch size
#         per_device_eval_batch_size=8,  # Evaluation batch size
#         logging_dir="./logs",  # Directory for logs
#         logging_steps=10,  # Log every 10 steps
#     )

#     # Initialize the model
#     def model_init():
#         return WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

#     # Set up the Trainer
#     trainer = Seq2SeqTrainer(
#         model_init=model_init,
#         args=training_args,
#         train_dataset=train_dataset,
#         eval_dataset=eval_dataset,
#         tokenizer="openai/whisper-small",  # Use the correct tokenizer for Whisper
#         data_collator=None,  # If needed, specify your data collator
#         compute_metrics=None,  # If you have a custom metric, specify it here
#     )

#     # Train the model
#     trainer.train()

#     # Evaluate and return the evaluation loss
#     eval_results = trainer.evaluate()
#     return eval_results["eval_loss"]

# # Create the Optuna study and run the optimization
# study = optuna.create_study(direction="minimize")  # Minimize the eval loss
# study.optimize(objective, n_trials=10)  # Try 10 different combinations

# # Print the best hyperparameters
# print("Best hyperparameters:", study.best_params)